# Basics

Mostly exercises and experiments made during
[Martin Odersky's functional programming course on Coursera](https://www.coursera.org/learn/progfun1)

## (Tail) recursion

In [3]:
import scala.annotation.tailrec

def fibonacci_tail_recursive(n: Long): BigInt = {
// @tailrec enforces method it annotates to be tail recursive
    @tailrec def fib(n: Long, acc: BigInt): BigInt = n match {
      case n if n >= 1 => fib(n - 1, acc * n)
      case _ => acc
    }

  fib(n, 1)
}

def fibonacci_not_tail_recursive(n: BigInt): BigInt = n match {
  case n if n > 1 => n * fibonacci_not_tail_recursive(n - 1)
  case _ => 1
}

fibonacci_tail_recursive(100)

fibonacci_not_tail_recursive(100) // should work if currect max stack size is > 1000

fibonacci_tail_recursive(50000)

// fibonacci_not_tail_recursive(50000) // will probably not work without changing jvm settings

try {
  (1000 to 20000 by 1000).map { i =>
    println(s"I is: $i")
    try {
      fibonacci_not_tail_recursive(i)
    } catch {
      case e: StackOverflowError =>
        throw new RuntimeException(
          s"""
             |Stack length that we reached was $i
             |see https://blogs.oracle.com/saas-fusion-app-performance/how-to-set-stack-size-to-overcome-javalangstackoverflowerror")
           """.stripMargin)
    }
  }
} catch {
  case e: RuntimeException =>
    println(e.getMessage)
}

def sum(i: Int, j: Int, f: Int => Int): Int = {
  if (i < j) f(i) + sum(i + 1, j, f)
  else f(j)
}

def sumtr(i: Int, j: Int, f: Int => Int): Int = {
  @tailrec def loop(acc: Int, i: Int, j: Int, f: Int => Int): Int = {
    if(i > j) acc
    else loop(acc + f(i), i + 1, j, f)
  }

  loop(0, i, j, f)
}
sumtr(1, 3, k => k * k)
sum(1, 3, k => k * k)

val params = Tuple3(1,5,(k:Int) => k * k * k)

assert(sum(1, 3, k => k * k) == sumtr(1, 3, k => k * k))
assert(sum(1,5, k => k * k * k) == sumtr(params._1, params._2, params._3))


I is: 1000
I is: 2000
I is: 3000
I is: 4000
I is: 5000

Stack length that we reached was 5000
see https://blogs.oracle.com/saas-fusion-app-performance/how-to-set-stack-size-to-overcome-javalangstackoverflowerror")
           


import scala.annotation.tailrec


defined function fibonacci_tail_recursive
defined function fibonacci_not_tail_recursive
res2_3: BigInt = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
res2_4: BigInt = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
res2_5: BigInt = 33473205095971448369154760940714864779127732238104548077301003219901680221443656416973812310719169308798480438190208299893616384743066693742630572845363784038325756282123359987268244078235972356040853854441373383753568565536371168327405166076155165921406156075461294201790567479665498629242220022541553510718159801615476451810616674970217996537474972541139338191638823500630307644256874857271394651081909874909643486268589229807870031031008962861154553979911612940652327396971497211031261142860733793509687837

#### Use assertions [http://wiki.c2.com/?UseAssertions](http://wiki.c2.com/?UseAssertions)

## Currying and higher order functions

*Example:* Sqrt using fixed point of a function

In [36]:
def operationOnInterval(op: (Int, Int) => Int, init: Int)(f: Int=>Int)(i: Int, j: Int): Int = {
  if (i > j) init
  else op(f(i), operationOnInterval(op, init)(f)(i + 1, j))
}

def product(f: Int => Int)(i: Int, j: Int) =
  operationOnInterval((i:Int, j:Int) => i * j, 1)(f)(i, j)

assert(product(k => k * k)(1,4) == 576)
assert(product(k => k)(1,4) == 24)

def isCloseEnough(x: Double, y: Double, tollerance: Double = 0.0001): Boolean = {
  Math.abs(x - y) / y < tollerance
}
assert(! isCloseEnough(1, 1.001, 0.0005))
assert(isCloseEnough(1, 1.001, 0.01))

@tailrec def fixedPoint(f: Double => Double)(init: Double): Double = {
  val next = f(init)
  println(next)
  if(isCloseEnough(next, init)) next
  else fixedPoint(f)(next)
}

fixedPoint(x => 1 + x / 2)(1)

// We can use averageDump whenever we use f, as returned function takes a param (currying)
def averageDump(f: Double => Double)(x: Double): Double = (f(x) + x) / 2

// here we use a fixed point method to define square root
// sqrt is y that y * y = x, so y = x / y - if we find a point where y = x/y, we are close :)

def sqrt(x: Double) = fixedPoint(averageDump((y:Double) => x / y))(1)

sqrt(2)

assert(isCloseEnough(sqrt(2), Math.sqrt(2)))

1.5
1.75
1.875
1.9375
1.96875
1.984375
1.9921875
1.99609375
1.998046875
1.9990234375
1.99951171875
1.999755859375
1.9998779296875
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899
1.5
1.4166666666666665
1.4142156862745097
1.4142135623746899


defined function operationOnInterval
defined function product
defined function isCloseEnough
defined function fixedPoint
res35_8: Double = 1.9998779296875
defined function averageDump
defined function sqrt
res35_11: Double = 1.4142135623746899

#### Exercise

 1. Define product method (tail recursive)
 1. Define factorial using product

## Classes

#### Use `require` to validate input

In [50]:
class Rational(x: Int, y: Int) {
    require(y != 0, "Denominator cannot be 0")
    
    private def gcd(a: Int, b: Int): Int = {
        if (b == 0) a else gcd(b, a % b)
    }
    val g = gcd(x, y)
    val numer = x / g
    val denom = y / g
    
    def this(x: Int) = this(x, 1)
    
    def + (that: Rational) =
        new Rational(
            this.numer * that.denom + that.numer * this.denom,
            this.denom * that.denom
        )
    
    def < (that: Rational) = this.numer * that.denom < that.numer * this.denom
    
    def * (i: Int) = new Rational(numer * i, denom)
    
    def max (that: Rational) = if(this < that) that else this
    
    def unary_- = new Rational(-numer, denom)
    
    def - (other: Rational) = this + -other
    
    override def toString = s"$numer / $denom"
}

// new Rational(1,0) // throws an IllegalArgumentException

val one = new Rational(1)
val two = new Rational(2)
val twoThirds = new Rational(2,3)
val half = new Rational(1,2)

// methods as infix operators
two + twoThirds
half + half
half - half

two < half


// Operator precedence in scala is predefiend 
// http://scala-lang.org/files/archive/spec/2.11/06-expressions.html#infix-operations
two max half * 2 + half

defined class Rational
one: wrapper.wrapper.Rational = 1 / 1
two: wrapper.wrapper.Rational = 2 / 1
twoThirds: wrapper.wrapper.Rational = 2 / 3
half: wrapper.wrapper.Rational = 1 / 2
res49_5: wrapper.wrapper.Rational = 8 / 3
res49_6: wrapper.wrapper.Rational = 1 / 1
res49_7: wrapper.wrapper.Rational = 0 / 1
res49_8: Boolean = false
res49_9: wrapper.wrapper.Rational = 2 / 1

## IntSet

In [42]:
abstract class IntSet {
    def contains(x: Int): Boolean
    def incl(x: Int): IntSet
    def union(other: IntSet): IntSet
}

object Empty extends IntSet {
    def contains(x: Int): Boolean = false
    def incl(x: Int): IntSet = new NonEmpty(Empty, x, Empty)
    override def toString = "."
    def union(other: IntSet): IntSet = other
}

class NonEmpty(val lTree: IntSet, val elem: Int, val rTree: IntSet) extends IntSet {
    def contains(x: Int): Boolean = {
        if (x < elem) lTree contains x
        else if(x > elem) rTree contains x
        else x == elem
    }
    
    def incl(x: Int): IntSet = {
//         println(s"Adding $x to $this")
        if (x < elem) new NonEmpty(lTree incl x, elem, rTree)
        else if (x > elem) new NonEmpty(lTree, elem, rTree incl x)
        else this
    }
    
    override def toString = s"(${lTree}_${elem}_${rTree})"
    
//     def union(other: IntSet): IntSet = {
//         println("Next iteration = we got", this, other)
//         val lrTree =  rTree union lTree
//         println("rTree union lTree",lrTree)
//         (lrTree union other) incl elem
//     }
    
    def union(other: IntSet): IntSet = other match {
        case Empty => this
        case o: NonEmpty =>
            val meWE = (this incl o.elem)
            (meWE union o.lTree) union o.rTree
        }
    
}

val set1 = new NonEmpty(Empty, 1, Empty)
val set2 = new NonEmpty(Empty, 2, Empty)
val set3 = new NonEmpty(Empty, 3, Empty)
val set4 = new NonEmpty(Empty, 4, Empty)

set1 union set2 union set3 
new NonEmpty(set1, 2, set3) union new NonEmpty(set4, 6, Empty)

defined class IntSet
defined object Empty
defined class NonEmpty
set1: wrapper.wrapper.NonEmpty = (._1_.)
set2: wrapper.wrapper.NonEmpty = (._2_.)
set3: wrapper.wrapper.NonEmpty = (._3_.)
set4: wrapper.wrapper.NonEmpty = (._4_.)
res41_7: wrapper.wrapper.IntSet = (._1_(._2_(._3_.)))
res41_8: wrapper.wrapper.IntSet = ((._1_.)_2_(._3_((._4_.)_6_.)))

Traits are essentially abstract classes - check with compiler stages `scalac -Xprint:-1`

In [43]:
Set[Nothing]()

res42: Set[Nothing] = Set()

In [119]:
trait List[T] {
    def isEmpty: Boolean
    def :: (head: T): List[T] = {
        new Cons[T](head, this)
    }
    def nth(n: Int): T
}
class Nil[T] extends List[T] {
    val isEmpty = true
    override def toString = "Nil"
    def nth(n: Int): Nothing = throw new IndexOutOfBoundsException("Nil.nth")
}
class Cons[T](val head: T, val tail: List[T]) extends List[T] {
    val isEmpty = false
    def nth(n: Int): T = if (n == 0) head else tail.nth(n - 1)
    override def toString = s"$head.$tail"
}

object List {
    def apply[T](): List[T] = {
        new Nil[T]()
    }
    def apply[T](x: T): List[T] = {
        new Cons[T](x, List[T]())
    }
    def apply[T](x: T, y: T): List[T] = {
        new Cons[T](x, List[T](y))
    }
    def apply[T](x: T, y: T, z: T): List[T] = {
        new Cons[T](x, List[T](y, z))
    }
}

val l = new Nil[Int].::(1).::(2)
l.nth(0)

List(1, 2,3)

defined trait List
defined class Nil
defined class Cons
defined object List
l: wrapper.wrapper.List[Int] = 2.1.Nil
res118_5: Int = 2
res118_6: wrapper.wrapper.List[Int] = 1.2.3.Nil

### Nat

In [108]:
import scala.annotation.tailrec

abstract class Nat {
    def isZero: Boolean
    def predecessor: Nat
    def successor: Nat = new Succ(this)
    def + (that: Nat): Nat
    def - (that: Nat): Nat
}

object Zero extends Nat {
    val isZero = true
    def predecessor = throw new RuntimeException("Zero.predecessor")
    def + (that: Nat): Nat = that
    def - (that: Nat): Nat = that match {
        case Zero => this
        case _ => throw new RuntimeException("Zero.-(other) which is not Zero")
    }
    override def toString = "Zero"
}

class Succ(x: Nat) extends Nat {
    val isZero = false
    def predecessor: Nat = x
    
    def + (that: Nat): Nat = new Succ(x + that)
    def - (that: Nat): Nat = that match {
        case Zero => this
        case nz => x - nz.predecessor
    }
    
    override def toString = {
        @tailrec def loop(acc: Int, nat: Nat): Int = nat match {
            case Zero => acc
            case nz => loop(acc + 1, nat.predecessor)
        }
        
        s"${loop(0, this)}"
    }
}

val z = Zero
val _1 = Zero.successor
val _2 = Zero.successor.successor

val _10 = _1 + _2 + _2 + _1 + z + _2 + _2

_10.predecessor.predecessor

import scala.annotation.tailrec


defined class Nat
defined object Zero
defined class Succ
z: wrapper.wrapper.Zero.type = Zero
_1: wrapper.wrapper.Nat = 1
_2: wrapper.wrapper.Nat = 2
_10: wrapper.wrapper.Nat = 10
res107_8: wrapper.wrapper.Nat = 8

Operators ending with : (such as 'cons' operator of the list) associate to the right, and they are method calls of the right hand operands

In [7]:
val l1 = 1 :: 2 :: Nil
val l2 = 1 :: (2 :: Nil)
val l3 = Nil.::(2).::(1)

assert (l1 == l2)
assert (l2 == l3)

l1: List[Int] = List(1, 2)
l2: List[Int] = List(1, 2)
l3: List[Int] = List(1, 2)

In [20]:
def myInit[T](xs: List[T]): List[T] = xs match {
    case Nil => throw new Error("Nil.init")
    case x :: Nil => Nil
    case x :: xs => x :: init(xs)
}

defined function myInit

In [21]:
val listForInit = (1 :: 2 :: 3 :: Nil)
assert( myInit(listForInit) == listForInit.init)

listForInit: List[Int] = List(1, 2, 3)

In [55]:
// as ys will be at the end of the resulting list, we need to match on xs itself and add elements one by one
def concat[T](xs: List[T], ys: List[T]): List[T] = xs match {
    case Nil => ys
    case x :: xs2 => x :: concat(xs2, ys)
}

defined function concat

In [28]:
val l1 = List(1,2)
val l2 = 3 :: 4 :: Nil
assert(concat(l1, l2) == (l1 ++ l2))
concat(l1, l2)

l1: List[Int] = List(1, 2)
l2: List[Int] = List(3, 4)
res27_3: List[Int] = List(1, 2, 3, 4)

In [54]:
def reverse[T](xs: List[T], acc: List[T] = List[T]()): List[T] = xs match {
    case Nil => acc
    case x :: xs2 => reverse(xs2, x::acc) // more obvious is reverse(xs2) ++ List(x), but N*N complexity 
}

defined function reverse

In [38]:
val l1 = 1::2::3::Nil
assert(reverse(l1) == l1.reverse)

l1: List[Int] = List(1, 2, 3)

In [56]:
def removeAt[T](xs: List[T], n: Int): List[T] = (xs, n) match {
    case (Nil, _) => xs // if out of bounds, return xs itself
    case (y :: ys, 0) => ys
    case (y :: ys, m) => y :: removeAt(ys, m - 1)
}

def removeAt2[T](xs: List[T], n: Int): List[T] = (xs take n) ++ (xs drop n+1)

defined function removeAt
defined function removeAt2

In [57]:
import scala.collection.mutable.ListBuffer
val l1 = 1::2::3::4::Nil

assert(removeAt(Nil, 2) == Nil)
assert(removeAt(Nil, 0) == Nil)
val lb1 = ListBuffer(l1:_*)
lb1.remove(2)
assert(removeAt(l1, 2) == lb1.toList)
assert(removeAt(l1, 2) == removeAt2(l1, 2))
assert(removeAt(l1,10) == l1)

import scala.collection.mutable.ListBuffer

l1: List[Int] = List(1, 2, 3, 4)
lb1: ListBuffer[Int] = ListBuffer(1, 2, 4)
res56_5: Int = 3

### Insertion sort

Algo:
 - take first elem
 - sort the rest
 - insert that first element where it fits
 
Complexity N*N

In [59]:
def insert[T](elem: T, xs: List[T], order: (T,T) => Boolean): List[T] = xs match {
    case Nil => elem :: Nil
    case y :: ys if order(y, elem) => y :: insert(elem, ys, order)
    case y :: ys => elem :: xs 
}

def isort[T](xs: List[T], order: (T,T) => Boolean): List[T] = xs match {
    case Nil => Nil
    case y :: ys => insert(y, isort(ys, order), order)
}

val l1 = List(4,2,6,7,3,6,9,14,0)
isort(l1, (x:Int, y:Int) => x < y)

defined function insert
defined function isort
l1: List[Int] = List(4, 2, 6, 7, 3, 6, 9, 14, 0)
res58_3: List[Int] = List(0, 2, 3, 4, 6, 6, 7, 9, 14)

### Merge sort

Algo:

* if list length <= 1 - return that list as sorted
* otherwise:
 - split list in half
 - sort first, sort second
 - merge both preserving order

In [62]:
def merge[T](xs: List[T], ys: List[T], order: (T, T) => Boolean): List[T] = xs match {
    case Nil => ys
    case z :: zs => insert(z, merge(zs, ys, order), order)
}
def msort[T](xs: List[T], order: (T, T) => Boolean): List[T] = xs match {
    case Nil => xs
    case x :: Nil => xs
    case _ => 
        val middle = xs.size / 2
        val ys = xs take middle
        val zs = xs drop middle
        merge(msort(ys, order), msort(zs, order), order)
}

defined function merge
defined function msort

In [63]:
val l1 = List(4,2,6,7,3,6,9,14,0)
msort(l1, (x:Int, y:Int) => x < y)

l1: List[Int] = List(4, 2, 6, 7, 3, 6, 9, 14, 0)
res62_1: List[Int] = List(0, 2, 3, 4, 6, 6, 7, 9, 14)

In [68]:
def pack[T](xs: List[T]): List[List[T]] = xs match {
    case Nil => Nil
    case y :: ys => (xs takeWhile (_ == y)) :: pack(xs dropWhile(_ == y))
}

defined function pack

In [69]:
val l1 = List("a","a","a","b","b","a","a","b","a","a","a")
pack(l1)

l1: List[String] = List("a", "a", "a", "b", "b", "a", "a", "b", "a", "a", "a")
res68_1: List[List[String]] = List(
  List("a", "a", "a"),
  List("b", "b"),
  List("a", "a"),
  List("b"),
  List("a", "a", "a")
)

In [72]:
def encode[T](xs: List[T]): List[(T, Int)] = {
    pack(xs).map { pack =>
        (pack.head, pack.size)
    }
}

defined function encode

In [78]:
encode(l1)

res77: List[(String, Int)] = List(("a", 3), ("b", 2), ("a", 2), ("b", 1), ("a", 3))

### Reductions

In [74]:
1::2::3::Nil reduceLeft (_ + _)

res73: Int = 6

In [81]:
def isPrime(n: Int): Boolean = (2 to n - 1).forall(div => n % div > 0)

defined function isPrime

In [84]:
isPrime(13)

res83: Boolean = true